In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MNISTConvNet(nn.Module):

    def __init__(self):
        # this is the place where you instantiate all your modules
        # you can later access them using the same names you've given them in
        # here
        super(MNISTConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    # it's the forward function that defines the network structure
    # we're accepting only a single input in here, but if you want,
    # feel free to use more
    def forward(self, input):
        x = self.pool1(F.relu(self.conv1(input)))
        x = self.pool2(F.relu(self.conv2(x)))

        # in your model definition you can go full crazy and use arbitrary
        # python code to define your model structure
        # all these are perfectly legal, and will be handled correctly
        # by autograd:
        # if x.gt(0) > x.numel() / 2:
        #      ...
        #
        # you can even do a loop and reuse the same module inside it
        # modules no longer hold ephemeral state, so you can use them
        # multiple times during your forward pass
        # while x.norm(2) < 10:
        #    x = self.conv1(x)

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [2]:
net = MNISTConvNet()
print(net)

MNISTConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [3]:
input = torch.randn(1, 1, 28, 28)
out = net(input)
print(out.size())

torch.Size([1, 10])


In [4]:
target = torch.tensor([3], dtype=torch.long)
loss_fn = nn.CrossEntropyLoss()  # LogSoftmax + ClassNLL Loss
err = loss_fn(out, target)
err.backward()

print(err)

tensor(2.3275, grad_fn=<NllLossBackward>)


In [22]:
net.conv1.weight.size()

torch.Size([10, 1, 5, 5])

In [23]:
net.conv1.weight.grad.size()

torch.Size([10, 1, 5, 5])

In [29]:
err.grad_fn

In [45]:
def printnorm(self, input, output):
    # input is a tuple of packed inputs
    # output is a Tensor. output.data is the Tensor we are interested
    print('Inside ' + self.__class__.__name__ + ' forward')
    print('')
    print('input: ', type(input))
    print('input: ', input)
    print('input len: ', len(input))
    print('input[0]: ', type(input[0]))
    print('output: ', type(output))
    print('')
    print('input size:', input[0].size())
    print('output size:', output.data.size())
    print('output norm:', output.data.norm())


# net.conv2.register_forward_hook(printnorm)

In [47]:
out = net(input)

Inside Conv2d forward

input:  <class 'tuple'>
input:  (tensor([[[[-0.4550,  0.2048,  1.2065, -1.0196,  0.5834, -0.2306,  1.9071,
           -0.0456,  0.4880,  1.4691,  0.8684, -2.0982, -1.1827, -2.1608,
            1.4475,  0.9553, -1.2447,  0.5514,  1.5241,  0.4140,  1.2250,
            0.4753,  0.2571,  0.2183, -2.4382, -0.9364, -0.4046,  0.1939],
          [-0.6125, -1.1245,  0.2525,  1.3134, -1.9491,  1.1396, -1.6473,
           -2.5672,  0.4604,  0.9389, -0.3131, -1.4419,  0.1234,  1.8143,
           -1.3185, -0.2337, -0.3829,  0.3574, -0.6424, -0.7010,  0.0533,
           -1.7722, -2.4586,  1.1290,  0.8116,  1.9008,  0.4258,  1.7755],
          [ 1.0200,  0.4956, -1.8514, -0.5587,  1.4646,  0.0534, -1.0389,
            0.3969, -1.0605,  0.3829, -0.2759, -1.1400,  1.5830, -0.0952,
            2.2370,  0.9062, -0.3771,  2.3235, -0.1593,  0.8163,  0.4470,
           -1.7632, -1.2523, -0.3382, -0.4972, -0.3634,  1.3637,  1.1882],
          [-1.6735,  1.1846, -0.2945,  0.9267, -1.119

In [46]:
net.conv1.register_forward_hook(printnorm)

In [54]:
for m in net.children():
    print(dir(m))
    break

['__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bias', 'buffers', 'children', 'conv2d_forward', 'cpu', 'cuda', 'dilation', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward', 'groups', 'half', 'in_channels', 'kernel_size', 'load_state_di

In [52]:
for m in net.modules():
    print(m)

MNISTConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Linear(in_features=320, out_features=50, bias=True)
Linear(in_features=50, out_features=10, bias=True)
